In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import re
import csv
import json
from datetime import datetime

In [26]:
BASE_PATH = '/content/drive/My Drive/devnagri-to-roman'
path_input = os.path.join(BASE_PATH, "data")
path_hindi = os.path.join(BASE_PATH, "hindi")
path_english = os.path.join(BASE_PATH, "english")
files = os.listdir(path_input)
print(files)

['454670_857975-val.txt', 'bbc-test.txt', 'bbc-train.txt', '454670_857975-train.txt', '454536_857792-train.txt', '454520_857761-valid.txt', '454536_857792-test.txt', '454520_857761-train.txt']


## Data Cleaning and Loading

In [0]:
b_list = []
for file_name in files:
    with open(os.path.join(path_input,file_name)) as file_read:
        data = file_read.read()
        data = re.sub('[0-9]',' ', data)
        data_list=data.replace('\n',' ').replace('/',' ').replace('.',' ').replace('(',' ').replace(')',' ').replace('?',' ').replace('-',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('\"',' ').replace('"',' ').replace('“',' ').replace('^',' ').replace('|',' ').replace('@',' ').replace('”',' ').replace('{',' ').replace('!',' ').replace('}',' ').split(' ') 
        data_list = [ word for word in data_list if len(word)<20 ]
        data_list = [ word for word in data_list if not word.isalnum() ]
        data_list = list(filter(None, data_list))    
    with open(os.path.join(path_hindi,file_name), 'w') as hindi_file:
        hindi_file.writelines("%s\n" % word for word in set(data_list))
    b_list.extend(set(data_list))


In [0]:
reader = csv.reader(open(os.path.join(BASE_PATH, "svar.csv"), 'r'))
vowels = {}
for row in reader:
	k, v = row
	vowels[k] = v

reader = csv.reader(open(os.path.join(BASE_PATH, "vyanjan.csv"), 'r'))
consonants = {}
for row in reader:
	k, v = row
	consonants[k] = v

## Unique Words

In [29]:
len(set(b_list))

602908

## Translitration

In [0]:
str1 = ""
for x in set(b_list):
	for y in x.split():
		for i in range(len(y)):
			if (i+1<len(y) and y[i+1].strip()==' ़'.strip()):
				c = y[i]+y[i+1]
				p=2
			else:
				c = y[i]
				p=1
			if (c in vowels.keys()):
				str1 = str1 + vowels[c]
			elif (c in consonants.keys()):
				if(i+p<len(y) and y[i+p] in consonants.keys()):
					if ((c=='झ' and i!=0) or (i!=0 and i+p+1<len(y) and y[i+p+1] in vowels.keys())): 
						str1 = str1 + consonants[c]
					else:
						str1 = str1 + consonants[c]+'a'
				else:
					str1 = str1 + consonants[c]
			elif y[i] in ['\n','\t',' ','!',',','।','-',':','\\','_'] or c.isalnum():
				str1 = str1 + c.replace('।','.')
		str1 = str1 + " "
	str1 = str1 + "\n"

In [0]:
with open(os.path.join(path_english,file_name), 'w') as eng_file:
    eng_file.write(str1.replace(' ','').strip())

In [0]:
word_dict = {}
hindi = set(b_list)    
english = [line.strip('\n') for line in open(os.path.join(path_english,file_name), 'r')]
word_dict.update(dict(zip(english, hindi))) 


In [0]:
timestamp = str(int(datetime.timestamp(datetime.now())))
with open(os.path.join(BASE_PATH,'data-'+timestamp+'.json'), 'w', encoding='utf-8') as final:
    json.dump(word_dict, final, ensure_ascii=False, indent=4)
